In [26]:
%load_ext autoreload
%autoreload 2
import glob
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib qt
import cartopy.crs as ccrs
import numpy as np
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
#import dictionaries from my_tools to automize plotting
from my_tools import plot_dic
from cartopy.util import add_cyclic_point
import matplotlib
#Set matplotlib to use agg so the plot doesn't constantly show up when I produce it which is very annoying.
matplotlib.use('agg')




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
#Set variable to plot 
variablez = ['u_wind','envelope','phasespeed']
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_mean/'

Normal 10 lead day error plot

In [38]:
#list all the relevant files
for variable in variablez:
    filez = sorted(glob.glob(path+'/*{}_day_??_globe_mars.nc'.format(plot_dic[variable]['filename'])))
    #create orthographic plot for each lead day
    latz = [(85,0),(0,-85)]
    for lat_hem in latz:
        if lat_hem==(85,0):
            fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,90)})
        else:
            fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,-90)})

        #Looping over all lead day error files
        for i, ax in enumerate(axz.flat):
            ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
            #Discard 85 - 90 degrees north, because these data yield unrealistical large errors
            ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
            ds_cyclic, lon_cyclic = add_cyclic_point(ds[plot_dic[variable]['data_variable']].values, coord=ds.lon)
            lon,lat = np.meshgrid(lon_cyclic,ds.lat)
            #Create coastlines
            ax.coastlines()
            #Plotting the data
            #Sometimes contourf function raises unexplainable ValueError (might be bug). Then I switch to pcolormesh
            try:
                im = ax.contourf(lon,lat,ds_cyclic,
                                transform=ccrs.PlateCarree(),
                                levels = np.linspace(-2,2,41),
                                cmap='bwr',
                                vmin=-2,vmax=2,
                                extend='both')
            except ValueError:
                im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds.v.values,
                                  transform = ccrs.PlateCarree(),cmap='bwr',
                                  vmin=-2,vmax=2)
            #Plotting the variable mean error below the graph
            #Only taking data between 80 and 30 degrees N because other data is not representative for mid-latitudes
            #For Souther Hemisphere we consider latitudes between -30 and -80 degrees N
            if lat_hem==(85,0):
                ds_text = ds.sel(lat=slice(80,30))
            else:
                ds_text = ds.sel(lat=slice(-30,-80))
            ax.text(x=0.5,y=-0.15,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area(ds_text))),
                   transform=ax.transAxes,horizontalalignment='center',fontsize=12)
            #set gridlines
            gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
            ax.set_title('Day {}'.format(i+1))

        #set figure size
        fig.set_size_inches(20, 10)
        fig.suptitle('GEFS mean {} error compared to ERA5'.format(plot_dic[variable]['title']))
        #Make room for color bar and create color bar
        fig.subplots_adjust(left=0.03,right=0.9)
        cbax = fig.add_axes([0.925, 0.2, 0.025, 0.6])
        fig.colorbar(im, cax=cbax)
        cbax.set_ylabel(plot_dic[variable]['label'], rotation=90)
        if lat_hem==(85,0): 
            fig.savefig(path+'/GEFS_forecast_mean_{}_error_NH.png'.format(plot_dic[variable]['filename']))
        else:
            fig.savefig(path+'/GEFS_forecast_mean_{}_error_SH.png'.format(plot_dic[variable]['filename']))
        plt.close(fig)

Simple 1 lead day error plot


In [10]:
#list all files
filez = glob.glob('/media/onno/Algemeen/Thesis/GFS_mean/*{}*CFS.nc'.format(plot_dic[variable]['filename']))

#Loop over all files
for i, file in enumerate(filez):
    #open associated file for lead day
    ds = xr.open_dataset(file,decode_times=False).squeeze()
    ds = ds.sel(lat=slice(80,20))
    #create figure
    fig,ax = plt.subplots(subplot_kw={'projection': ccrs.Orthographic(0,90)})
    ax.coastlines()
    #plot data
    im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds[plot_dic[variable]['data_variable'].values,
                    transform=ccrs.PlateCarree(),
                    cmap='bwr',
                    vmin=-2,vmax=2)
    
    #create colorbar
    cbar = fig.colorbar(im,boundaries = (-2,2))
    cbar.set_label(plot_dic[variable]['label'], rotation=90)
    fig.suptitle('GEFS mean RWP {} error day {}'.format(plot_dic[variable]['title'],i))
    fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean/GEFS_forecast_{}_error_day_{:02d}'.format(plot_dic[variable]['filename'],i))
    plt.close(fig)

SyntaxError: invalid syntax (<ipython-input-10-a1e5af80ea56>, line 14)

Normal 10 day plot for each season

In [44]:
#Loop over all seasons
path = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
variablez = ['phasespeed']
seasons = ['JJA']
for variable in variablez:
    for season in seasons:
        latz = [(85,0),(0,-85)]
        for lat_hem in latz:
            #list all files for each season
            filez = sorted(glob.glob(path+'/*{}_{}*globe.nc'.format(plot_dic[variable]['filename'],season)))
            #create orthographic plot for each lead day
            if lat_hem==(85,0):
                fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,90)})
            else:
                fig,axz = plt.subplots(2,5,subplot_kw={'projection': ccrs.Orthographic(0,-90)})
            fig.set_size_inches(20, 10)
            #Plot each mean error for each lead day
            for i, ax in enumerate(axz.flat):
                #open accompanying netcdf file for lead day error
                ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
                ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
                ds_cyclic, lon_cyclic = add_cyclic_point(ds[plot_dic[variable]['data_variable']].values, coord=ds.lon)
                lon,lat = np.meshgrid(lon_cyclic,ds.lat)
                #Create coastlines
                ax.coastlines()
                #Plotting data
                #Sometimes contourf function raises unexplainable ValueError (might be bug). Then I switch to pcolormesh
#                 try:
                im = ax.contourf(lon,lat,ds_cyclic,
                                transform=ccrs.PlateCarree(),
                                levels = np.linspace(-2,2,41),
                                cmap='bwr',
                                vmin=-2,vmax=2,
                                extend='both')
#                 except ValueError:
#                 im = ax.pcolormesh(ds.lon.values,ds.lat.values,ds.v.values,
#                                   transform = ccrs.PlateCarree(),cmap='bwr',
#                                   vmin=-2,vmax=2)
                #plot text with mean error belowhemisphere plot
                #Only taking data between 80 and 30 degrees N because other data is not representative for mid-latitudes
                #For Souther Hemisphere we consider latitudes between -30 and -80 degrees N
                if lat_hem==(85,0):
                    ds_text = ds.sel(lat=slice(80,30))
                else:
                    ds_text = ds.sel(lat=slice(-30,-80))
                ax.text(x=0.5,y=-0.15,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area(ds_text))),
                       transform=ax.transAxes,horizontalalignment='center',fontsize=12)
                gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
                ax.set_title('Day {}'.format(i+1))

            # cbar = fig.colorbar(im)
            # cbar.set_label('E (m/s)', rotation=90)
            fig.suptitle('GEFS mean RWP {} error {} compared to ERA5'.format(plot_dic[variable]['title'],season))
            #Set colorbar and make room for it
            fig.subplots_adjust(right=0.85)
            cbax = fig.add_axes([0.875, 0.2, 0.025, 0.6])
            fig.colorbar(im, cax=cbax)
            cbax.set_ylabel(plot_dic[variable]['label'], rotation=90)
            #Save figure and make sure it does not open a window
            if lat_hem == (85,0):
                fig.savefig(path+'/GEFS_forecast_{}_error_{}_NH.png'.format(plot_dic[variable]['filename'],season),dpi=300)
            else:
                fig.savefig(path+'/GEFS_forecast_{}_error_{}_SH.png'.format(plot_dic[variable]['filename'],season),dpi=300)
            plt.close(fig)

ValueError: Sequences of multi-polygons are not valid arguments

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f3081bf80d0> (for post_execute):


ValueError: Sequences of multi-polygons are not valid arguments